## Local Function Review

In [1]:
def outer_function():
    message = "Hi"
    def inner_function():
        print(message)
    return inner_function()

In [2]:
# Test it
outer_function()

Hi


In [3]:
def outer_function():
    message = "Hi"
    def inner_function():
        print(message)
    return inner_function  # return the function

In [4]:
my_func = outer_function()
my_func

<function __main__.outer_function.<locals>.inner_function>

In [5]:
my_func()

Hi


In [6]:
# now add some variables
def outer_function(msg):   # added msg
    message = msg
    def inner_function():
        print(message)
    return inner_function  # return the function

In [8]:
hi_func = outer_function("Hi")
bye_func = outer_function("Bye")

# Run it
hi_func()

Hi


In [9]:
bye_func()

Bye


In [10]:
def outer_function(message):
    def inner_function():        # eliminated unnecessary local
        print(message)
    return inner_function

In [12]:
hi_func = outer_function("Hi")
bye_func = outer_function("Bye")

# Run it
hi_func()

Hi


In [13]:
bye_func()

Bye


## Decorators
- A function that takes another function as an argument
- Adds some functionality
- Returns another function
- All of this without altering the original function

In [14]:
def decorator_function(original_function):
    def wrapper_function():
        return(original_function())
    return(wrapper_function)

def display():
    print("Display function ran")

In [15]:
# Test it
decorated_display = decorator_function(display)
decorated_display

<function __main__.decorator_function.<locals>.wrapper_function>

In [16]:
decorated_display()

Display function ran


In [17]:
def decorator_function(original_function):
    def wrapper_function():
        print("Wrapper ran before {}".format(original_function.__name__))
        return(original_function())
    return(wrapper_function)

def display():
    print("Display function ran")

In [18]:
# Test it
decorated_display = decorator_function(display)
decorated_display

<function __main__.decorator_function.<locals>.wrapper_function>

In [19]:
decorated_display()

Wrapper ran before display
Display function ran


## Now decorate the functions

**@Decorator_Function_Name**

In [20]:
def decorator_function(original_function):
    def wrapper_function():
        print("Wrapper ran before {}".format(original_function.__name__))
        return(original_function())
    return(wrapper_function)

@decorator_function
def display():
    print("Display function ran")

In [24]:
# Test it
display

<function __main__.decorator_function.<locals>.wrapper_function>

In [25]:
display()

Wrapper ran before display
Display function ran


This will not work if our original function takes arguments

In [28]:
def display_info(name, age):
    print("display_info ran with arguments ({}, {})".format(name, age))

display_info("Mario", 21)

display_info ran with arguments (Mario, 21)


In [30]:
@decorator_function
def display_info(name, age):
    print("display_info ran with arguments ({}, {})".format(name, age))
    
display_info("Joe", 55)

TypeError: wrapper_function() takes 0 positional arguments but 2 were given

We can fix this with **\*args** and **\*\*kwargs**

In [31]:
def decorator_function(original_function):
    def wrapper_function(*args, **kwargs):
        print("Wrapper ran before {}".format(original_function.__name__))
        return(original_function(*args, **kwargs))
    return(wrapper_function)

@decorator_function
def display():
    print("Display function ran")

@decorator_function
def display_info(name, age):
    print("display_info ran with arguments ({}, {})".format(name, age))

In [32]:
display()

Wrapper ran before display
Display function ran


In [33]:
display_info("Wookie", 33)

Wrapper ran before display_info
display_info ran with arguments (Wookie, 33)


### Example: escape unicode

In [37]:
def escape_unicode(f):
    def wrap(*args, **kwargs):
        return(ascii(f(*args, **kwargs)))
    return wrap

@escape_unicode    
def mexico_city():
    return("México")

mexico_city()

"'M\\xe9xico'"

### What can be a Decorator?
- Class Objects, which can be callable with the **dunder call**
- Functions as decorators

# Classes as Decorators

In [39]:
class DecoratorClass(object):
    def __init__(self, original_function):
        self.original_function = original_function
    def __call__(self, *args, **kwargs):
        print("Call method executed before {}".format(self.original_function))
        return(self.original_function(*args, **kwargs))
    
@DecoratorClass
def display():
    print("Display function ran")

@DecoratorClass
def display_info(name, age):
    print("display_info ran with arguments ({}, {})".format(name, age))

In [40]:
display()

Call method executed before <function display at 0x000001A238A29598>
Display function ran


In [41]:
display_info("Solo", 99)

Call method executed before <function display_info at 0x000001A238A29A60>
display_info ran with arguments (Solo, 99)


### Instances of Decorators
A class instance.

In [49]:
class Trace:
    def __init__(self):
        self._enable = True
    def __call__(self, f):
        def wrap(*args, **kwargs):
            if self._enable:
                print("Calling {}".format(f.__name__))
            return(f(*args, **kwargs))
        return(wrap)

# Create an instance
trace = Trace()

@trace
def rotate_list(myList):
    return(myList[1:] + [myList[0]])

Unlike our previous example the class object itself is **not** the decorator, rather an instance of the class is!

In [50]:
# Test it
theList = [1,2,3]
theRotatedList = rotate_list(theList)

Calling rotate_list


In [51]:
# turn off the enable key from the instance

trace._enable = False

# Test it
theList = [1,2,3]
theRotatedList = rotate_list(theList)

# Back to function decorators

In [61]:
import time

def my_timer(original_function):
    """
    Helps you keep track of the logging part of a function
    """
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_function(*args, **kwargs)
        t2 = time.time()
        print("Function: {} Time: {}".format(original_function, t2-t1))
        return(result)
    return(wrapper)

# Test it
@my_timer
def display_info(name, age):
    time.sleep(1)
    print("Display info ran ({}, {})".format(name, age))


In [62]:
# Test it
display_info("what", 10)

Display info ran (what, 10)
Function: <function display_info at 0x000001A238B14620> Time: 1.0002541542053223


## Multiple Decorators

In [83]:
def my_logger(f):
    import logging
    logging.basicConfig(filename="{}.log".format(f.__name__), level=logging.INFO)
    def wrapper(*args, **kwargs):
        logging.info("Ran with args: {}, and kwargs: {}".format(args, kwargs))
        return(f(*args, **kwargs))
    return(wrapper)

@my_logger
def display_info(name, age):
    time.sleep(1)
    print("Display info ran ({}, {})".format(name, age))

In [84]:
# Test it
display_info("Weber", 125)

Display info ran (Weber, 125)


In [86]:
# Test it with multiple decorators
@my_timer
@my_logger
def display_info(name, age):
    time.sleep(1)
    print("Display info ran ({}, {})".format(name, age))

display_info("mario", 21)

Display info ran (mario, 21)
Function: <function my_logger.<locals>.wrapper at 0x000001A238B91E18> Time: 1.0009534358978271


In [87]:
# Test it with multiple decorators (switch order of decorators)
@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print("Display info ran ({}, {})".format(name, age))

display_info("george", 36)

Display info ran (george, 36)
Function: <function display_info at 0x000001A238B7E950> Time: 1.000154733657837


Wrap everything

In [89]:
import time
from functools import wraps

def my_logger(f):
    import logging
    logging.basicConfig(filename="{}.log".format(f.__name__), level=logging.INFO)
    @wraps(f)
    def wrapper(*args, **kwargs):
        logging.info("Ran with args: {}, and kwargs: {}".format(args, kwargs))
        return(f(*args, **kwargs))
    return(wrapper)

def my_timer(original_function):
    @wraps(original_function)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_function(*args, **kwargs)
        t2 = time.time()
        print("Function: {} Time: {}".format(original_function, t2-t1))
        return(result)
    return(wrapper)

In [90]:
# Test it with multiple decorators
@my_logger
@my_timer
def display_info(name, age):
    time.sleep(1)
    print("Display info ran ({}, {})".format(name, age))

display_info("wraps", 107)

Display info ran (wraps, 107)
Function: <function display_info at 0x000001A238A296A8> Time: 1.0004613399505615
